In [ ]:
import gc
import math
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, GRU, LSTM, Dropout, TimeDistributed, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import  SGD, RMSprop, Adam
from functools import cmp_to_key
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new')

edges = [[1, 2], [1, 5], [1, 8], [2, 3], [3, 4], [3, 8], [3, 9], [4, 10], [5, 6], [6, 7], [6, 8], [6, 9], [7, 10], [9, 10]]
number_edges = len(edges)
number_steps = 4
start_node = 1
end_node = 10

df = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU.csv')
x = df.iloc[:,:number_edges].values
y = df.iloc[:,number_edges:].values

df_hp = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU_hp.csv')
x_hp = df_hp.iloc[:,:number_edges].values
y_hp = df_hp.iloc[:,number_edges:].values

from sklearn.model_selection import train_test_split
x_main_train,x_main_tv,y_main_train,y_main_tv = train_test_split(x,y,test_size=2/5)
x_main_test,x_main_valid,y_main_test,y_main_valid = train_test_split(x_main_tv,y_main_tv,test_size=1/2)

x_hp_train,x_hp_tv,y_hp_train,y_hp_tv = train_test_split(x_hp,y_hp,test_size=2/5)
x_hp_test,x_hp_valid,y_hp_test,y_hp_valid = train_test_split(x_hp_tv,y_hp_tv,test_size=1/2)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_main_train)
x_test = sc.transform(x_main_test)
x_valid = sc.transform(x_main_valid)

sc_hp = StandardScaler()
x_train_1 = sc_hp.fit_transform(x_hp_train)
x_test_1 = sc_hp.transform(x_hp_test)
x_valid_1 = sc_hp.transform(x_hp_valid)

x_train = np.repeat(x_train[:, np.newaxis, : ], number_steps, axis=1)
x_test = np.repeat(x_test[:, np.newaxis, : ], number_steps, axis=1)
x_valid = np.repeat(x_valid[:, np.newaxis, : ], number_steps, axis=1)

x_train_1 = np.repeat(x_train_1[:, np.newaxis, : ], number_steps, axis=1)
x_test_1 = np.repeat(x_test_1[:, np.newaxis, : ], number_steps, axis=1)
x_valid_1 = np.repeat(x_valid_1[:, np.newaxis, : ], number_steps, axis=1)

y_train = np.reshape(y_main_train,(y_main_train.shape[0], number_steps, number_edges))
y_test = np.reshape(y_main_test,(y_main_test.shape[0], number_steps, number_edges))
y_valid = np.reshape(y_main_valid,(y_main_valid.shape[0], number_steps, number_edges))

y_train_1 = np.reshape(y_hp_train,(y_hp_train.shape[0], number_steps, number_edges))
y_test_1 = np.reshape(y_hp_test,(y_hp_test.shape[0], number_steps, number_edges))
y_valid_1 = np.reshape(y_hp_valid,(y_hp_valid.shape[0], number_steps, number_edges))

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape, x_train_1.shape, y_train_1.shape, x_test_1.shape, y_test_1.shape)


Mounted at /content/gdrive
(300000, 4, 14) (300000, 4, 14) (100000, 4, 14) (100000, 4, 14) (100000, 4, 14) (100000, 4, 14) (6000, 4, 14) (6000, 4, 14) (2000, 4, 14) (2000, 4, 14)


In [ ]:
# Hyperparameters of structure: number of layers, number of units, dropout, cell type
# Hyperparameters of training: learning rate, batch size
HP_1 = [[1, 2, 3, 4, 5],
       [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, -1],
       [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
       [SimpleRNN, GRU, LSTM],
       [SGD, RMSprop, Adam],
       [0.0001, 0.001, 0.01, 0.1],
       [16, 32, 64, 128, 256, 512, 1024]]

In [ ]:
t = []
for p in HP_1:
    t.append(len(p))
t1 = []
hp = []
for i in range(t[0]):
    t1.append(t[1])
    t1.append(t[2])
    hp.append(HP_1[1])
    hp.append(HP_1[2])
for i in range(3, len(HP_1)):
    t1.append(t[i])
    hp.append(HP_1[i])
t2 = [int(np.sqrt(itm)) for itm in t1]
n_hp = 2*t[0] + len(HP_1) - 3
print(t, t1, t2, n_hp)

[5, 22, 6, 3, 3, 4, 7] [22, 6, 22, 6, 22, 6, 22, 6, 22, 6, 3, 3, 4, 7] [4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 1, 1, 2, 2] 14


In [ ]:
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, ratio=0.0, patience=0, verbose=0):
        super(CustomEarlyStopping, self).__init__()

        self.ratio = ratio
        self.patience = patience
        self.verbose = verbose
        self.stopped_epoch = 0
        self.val_loss_1 = 0
        self.rt = 0

    def on_epoch_end(self, epoch, logs=None):
        if epoch == 1:
            self.val_loss_1 = logs.get('val_loss')
        if epoch == self.patience:
            current_val = logs.get('val_loss')
            if current_val/self.val_loss_1 > self.ratio:
                self.rt = current_val/self.val_loss_1
                self.stopped_epoch = epoch
                self.model.stop_training = True

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0 and self.verbose > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch))
            print("Current ratio = ", self.rt)

In [ ]:
class Network:
    def __init__(self):
        self.gen = self.create_solution()
        self.fitness = float("inf")

    def create_solution(self):
        gen = []
        for i in range(n_hp):
            gen.append(np.random.choice(list(range(t1[i]))))
        return gen

    def check_gen(self):
        state = False
        for i in range(n_hp):
            if i%2 == 0 and i < 2*t[0] and hp[i][self.gen[i]] != -1:
                state = True
                break
        return state

    def evaluate(self):
        if self.check_gen() == False:
            return float("inf")
        else:
            input_shape = (number_steps, number_edges)
            inputs = Input(shape=input_shape)
            x_input = inputs
            cell_type = hp[2*t[0]][self.gen[2*t[0]]]
            for i in range(t[0]):
                if hp[2*i][self.gen[2*i]] != -1:
                    x_input = cell_type(hp[2*i][self.gen[2*i]], return_sequences=True)(x_input)
                    x_input = Dropout(hp[2*i+1][self.gen[2*i+1]])(x_input)
            outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
            model = Model(inputs=inputs, outputs=outputs)
            opt = hp[2*t[0]+1][self.gen[2*t[0]+1]]
            model.compile(loss=BinaryCrossentropy(), optimizer=opt(learning_rate=hp[2*t[0]+2][self.gen[2*t[0]+2]]))
            callback_1 = CustomEarlyStopping(ratio=0.9, patience=10, verbose=1)
            callback_2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
            model.fit(x_train_1, y_train_1, epochs=100, callbacks=[callback_1, callback_2], validation_data=(x_valid_1, y_valid_1), batch_size=hp[2*t[0]+3][self.gen[2*t[0]+3]], verbose=0)
            loss = model.evaluate(x_test_1, y_test_1, verbose=0)
            tf.keras.backend.clear_session()
            return loss

In [ ]:
class FA:
    def __init__(self, N, Max, y, a0, b0):
        self.N = N
        self.Max = Max
        self.population = [Network() for i in range(self.N)]
        self.best = Network()
        self.pool = []
        self.y = y
        self.a0 = a0
        self.b0 = b0
        self.a = 0

    def population_fitness(self):
        for i in range(len(self.population)):
            self.calculate_fitness(self.population[i])
            self.show(self.population[i])
        print("__________________________________________________________")

    def calculate_fitness(self, new_mem):
        cond = True
        for mem in self.pool:
            if tuple(new_mem.gen) == tuple(mem.gen):
                cond = False
                new_mem.fitness = mem.fitness
                break
        if cond == True:
            new_mem.fitness = new_mem.evaluate()
            self.pool.append(copy.deepcopy(new_mem))

    def complexity(self, m):
        coeff = None
        r_a = 0
        if m.gen[2*t[0]] == 0:
            coeff = 1
        elif m.gen[2*t[0]] == 1:
            coeff = 3
            r_a = 1
        else:
            coeff = 4
        a = 0
        g = number_edges
        for i in range(t[0]):
            if hp[2*i][m.gen[2*i]] != -1:
                b = coeff*hp[2*i][m.gen[2*i]]*(hp[2*i][m.gen[2*i]] + g + r_a + 1)
                a += b
                g = hp[2*i][m.gen[2*i]]
        a += number_edges*(g+1)
        return a

    def compare(self, m1, m2):
        if abs(m1.fitness-m2.fitness) < 0.01*min(m1.fitness, m2.fitness):
            a = self.complexity(m1)
            b = self.complexity(m2)
            if a < b:
                return -1
            elif a == b:
                if m1.fitness < m2.fitness:
                    return -1
                elif m1.fitness == m2.fitness:
                    return 0
                else:
                    return 1
            else:
                return 1
        else:
            if m1.fitness < m2.fitness:
                return -1
            else:
                return 1

    def attract(self):
        for i in range(self.N):
            for j in range(self.N):
                newSolution = Network()
                if self.compare(self.population[i], self.population[j]) == 1:
                    r2 = 0
                    for j1 in range(n_hp):
                        number = (self.population[i].gen[j1] - self.population[j].gen[j1])/t1[j1]
                        r2 += pow(number,2)
                    r2 = r2/n_hp

                    b = self.b0*math.exp(-self.y*r2)

                    for j1 in range(n_hp):
                        e = random.uniform(0, 1)-1/2
                        ax = round(self.population[i].gen[j1] + b*(self.population[j].gen[j1] - self.population[i].gen[j1]) + t1[j1]*self.a*e)
                        newSolution.gen[j1] = max(0, min(ax, t1[j1]-1))

                    self.calculate_fitness(newSolution)
                    self.population[i] = copy.deepcopy(newSolution)

    def current_population(self):
        for i in range(len(self.population)):
            self.show(self.population[i])
        print("__________________________________________________________________")

    def get_best(self):
        self.population.sort(key=cmp_to_key(self.compare))
        self.current_population()
        if self.compare(self.best, self.population[0]) == 1:
            self.best = copy.deepcopy(self.population[0])
        self.show(self.best)
        print("__________________________________________________________________")

    def show(self, m):
        p_copy = []
        for j in range(t[0]):
            p_copy.append((m.gen[2*j], m.gen[2*j+1]))
        p_copy.append(m.gen[2*t[0]:])
        print(p_copy, m.fitness)

    def do(self):
        self.population_fitness()
        for itr in range(self.Max):
            print("iteration = ", itr)
            self.a = self.a0*pow(0.9,itr)
            self.attract()
            self.current_population()
            self.get_best()
            gc.collect()

In [ ]:
N = 10
Max = 7
y = 1
a0 = 1
b0 = 1

In [ ]:
alg = FA(N, Max, y, a0, b0)
for i in range(alg.N):
  alg.show(alg.population[i])

[(6, 4), (14, 0), (10, 4), (13, 5), (11, 5), [2, 2, 0, 2]] inf
[(21, 0), (12, 4), (18, 3), (4, 3), (15, 0), [1, 1, 2, 6]] inf
[(5, 3), (14, 4), (11, 5), (8, 5), (20, 2), [1, 0, 2, 5]] inf
[(1, 5), (10, 2), (9, 0), (13, 3), (5, 0), [2, 2, 0, 0]] inf
[(17, 5), (18, 4), (18, 3), (15, 5), (17, 4), [2, 0, 0, 1]] inf
[(14, 5), (13, 5), (18, 0), (1, 1), (11, 5), [2, 2, 2, 2]] inf
[(20, 1), (13, 4), (17, 0), (5, 1), (21, 4), [0, 1, 0, 3]] inf
[(18, 2), (8, 3), (20, 5), (13, 5), (19, 1), [0, 0, 2, 6]] inf
[(19, 4), (19, 2), (21, 1), (6, 1), (7, 3), [1, 1, 3, 0]] inf
[(20, 5), (8, 3), (20, 0), (13, 1), (7, 4), [2, 1, 3, 2]] inf


In [ ]:
import time
start = time.time()
alg.do()
end = time.time()

Epoch 00010: early stopping
Current ratio =  0.9196372729128586
[(6, 4), (14, 0), (10, 4), (13, 5), (11, 5), [2, 2, 0, 2]] 0.583311140537262
Epoch 63: early stopping
[(21, 0), (12, 4), (18, 3), (4, 3), (15, 0), [1, 1, 2, 6]] 0.2829831540584564
Epoch 00010: early stopping
Current ratio =  0.9880355512879478
[(5, 3), (14, 4), (11, 5), (8, 5), (20, 2), [1, 0, 2, 5]] 0.6829908490180969
Epoch 00010: early stopping
Current ratio =  0.9661151071158944
[(1, 5), (10, 2), (9, 0), (13, 3), (5, 0), [2, 2, 0, 0]] 0.578104555606842
Epoch 00010: early stopping
Current ratio =  0.9988606727294888
[(17, 5), (18, 4), (18, 3), (15, 5), (17, 4), [2, 0, 0, 1]] 0.6921855211257935
Epoch 41: early stopping
[(14, 5), (13, 5), (18, 0), (1, 1), (11, 5), [2, 2, 2, 2]] 0.3235015571117401
[(20, 1), (13, 4), (17, 0), (5, 1), (21, 4), [0, 1, 0, 3]] 0.3556671142578125
Epoch 00010: early stopping
Current ratio =  0.9775607434539286
[(18, 2), (8, 3), (20, 5), (13, 5), (19, 1), [0, 0, 2, 6]] 0.7087006568908691
Epoch 0001

KeyboardInterrupt: 

In [ ]:
alg.best.gen

In [ ]:
end = time.time()

In [ ]:
run_time = end - start
round(run_time)

23178

In [ ]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
# input_shape = (number_steps, number_edges)
# inputs = Input(shape=input_shape)
# x_input = inputs
# cell_type = hp[2*t[0]][alg.best.gen[2*t[0]]]
# for i in range(t[0]):
#     if hp[2*i][alg.best.gen[2*i]] != -1:
#         x_input = cell_type(hp[2*i][alg.best.gen[2*i]], return_sequences=True)(x_input)
#         x_input = Dropout(hp[2*i+1][alg.best.gen[2*i+1]])(x_input)
# outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
# model = Model(inputs=inputs, outputs=outputs)
# opt = hp[2*t[0]+1][alg.best.gen[2*t[0]+1]]
# model.compile(loss=BinaryCrossentropy(), optimizer=opt(learning_rate=hp[2*t[0]+2][alg.best.gen[2*t[0]+2]]))
# model.summary()
# print(alg.complexity(alg.best))
# print(opt)
# history = model.fit(x_train, y_train, epochs=1000, batch_size=hp[2*t[0]+3][alg.best.gen[2*t[0]+3]], callbacks=[callback], validation_data=(x_valid, y_valid), verbose=1)

In [ ]:
# from Results import Results
# pred = model.predict(x_train)
# pred_test = model.predict(x_test)
# r = Results(number_edges, edges, start_node, end_node, number_steps)
# accuracy_train = r.get_accuracy(pred, x_main_train, y_train)
# accuracy_test = r.get_accuracy(pred_test, x_main_test, y_test)
# print(accuracy_train, accuracy_test)